# Configurations

In [45]:
import config_file;
configs = config_file.readConfigs();
configs['camera_url']

'http://192.168.0.103:8080/shot.jpg'

# Main Classes

In [46]:
import numpy as np;
import pandas as pd;
import cv2;
import math;
from tqdm.notebook import tqdm;
from time import sleep;
import termcolor;
from ipdb import set_trace;
import PIL;
import IPython;
from io import BytesIO;

class VideoStreamBase():

    def logPeople(self, people, tolerance=configs['tolerance']):

        def isPresent1(id, i):
            if(i >= len(people)): return False;
            return id in [id_ for _, id_, _ in people[i]];
        
        def isPresent(id, i, j):
            for k in range(i, j + 1):
                if(isPresent1(id, k)):
                    return True;
            return False;
        
        def getLength():
            ret = 0;
            for frame in people:
                for (_, id, _) in frame:
                    if(id is None): continue;
                    ret = max(ret, id);
            return ret + 1;
        

        data = pd.DataFrame(columns=['face_id', 'person_id', 'name', 'timestamp_first', 'timestamp_last']);

        idx = 0;
        entryTime = [None] * getLength();
        for i, frame in enumerate(people):
            for j, person in enumerate(frame):
                id = person[1];

                if(id is None):
                    continue;

                if(entryTime[id] is None):
                    entryTime[id] = (i, person);

                if(isPresent(id, i + 1, i + tolerance)):
                    continue;
                
                data.at[idx] = list(person) + [entryTime[id][0], i];
                idx += 1;
                entryTime[id] = None;
                
        for id, el in enumerate(entryTime):
            if(el is None): continue;
            data.at[idx] = list(el[1]) + [el[0], len(people) - 1];
            idx += 1;

        data.to_csv('./database/Presence.csv');
        return data;

    def __del__(self):
        cv2.VideoCapture(0).release();
        cv2.destroyAllWindows();

    def getImage(self) -> np.ndarray:
        raise NotImplementedError();

    def displayVideo(self, limit: int = 0, lag: int = 0, display_id: int = 1, width: int = 400, height: int = 200):
        try:
            while(self.displayImage(display_id, width, height)):
                sleep(lag);
        except(KeyboardInterrupt):
            print(termcolor.colored('video stream stopped voluntarily', 'green'));

    def displayImage(self, display_id: int = None, width: int = 400, height: int = 200, img = None) -> bool:
        img = self.getImage() if img is None else img;
        if(img is None): return False;

        img = PIL.Image.fromarray(img, 'RGB')
        buffer = BytesIO()
        img.save(buffer, format="jpeg")
        display(
            IPython.display.Image(
                data=buffer.getvalue(), format='jpeg', width=width, height=height
                ), display_id=display_id);

        return True;

    def __len__(self):
        raise NotImplementedError();

    def fps(self):
        raise NotImplementedError();

    def download(self, path: str = './output.mp4', fps: float = None, limit: int = None, display: bool = False,
                 safeMode: bool = True, width: int = None, height: int = None):

        nFrame = 1 if len(self) == 1 else (len(self) - 1 if limit is None else limit);
        if(nFrame == 0): raise Exception('No Frames Found');

        people = [[]];
        try:    img = self.getImage(people[-1]);
        except: img = self.getImage();

        if(height is None): height = img.shape[0];
        if(width  is None): width  = img.shape[1];

        fourcc =  cv2.VideoWriter_fourcc(*'XVID');
        fps_ = self.fps() if fps is None else fps;
        outputVideo = cv2.VideoWriter(path, fourcc, fps_, (width, height), 1)



        with tqdm(total=nFrame) as progressbar:
            currentFrame = 0;
            nextImg = None;
            while(currentFrame < nFrame and img is not None):
                try:
                    if(display): self.displayImage(1, width, height, img);
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB);

                    people.append([]);
                    try:    nextImg = self.getImage(people[-1]);
                    except: nextImg = self.getImage();

                except Exception as e:
                    if(safeMode):
                        print(termcolor.colored('Exception:', 'red'));
                        print(e);
                    else:
                        raise e;

                outputVideo.write(img);
                currentFrame += 1;
                progressbar.update(1);
                img = nextImg;

        outputVideo.release();
        return self.logPeople(people);

In [47]:
import face_recognition;
import os;
import json;


class FaceDetector(VideoStreamBase):

    video: VideoStreamBase

    def __len__(self):
        return len(self.video);

    def fps(self):
        return self.video.fps();

    def __init__(self, video: VideoStreamBase):
        self.video = video;

        self.known_faces = None;
        with open(configs['db_face'], 'r') as f:
            embeddings = json.load(f);

            self.known_faces = [None] * len(embeddings);
            for el in embeddings:
                self.known_faces[el['id']] = np.array(el['embedding']);

        self.known_names = None;
        with open(configs['db_person'], 'r') as f:
            people = json.load(f);

            self.known_names = [None] * len(people);
            assert(len(self.known_names) == len(self.known_faces))

            for el in people:
                self.known_names[el['face']] = (el['id'], el['name']);

    def getImage(self, people):
        img = self.video.getImage();
        if(img is None): return None;

        face_locations = face_recognition.face_locations(img, model=configs['face_locations_model'])
        face_encodings = face_recognition.face_encodings(img, face_locations, model=configs['face_encodings_model'])
        # set_trace();


        for face_encoding in face_encodings:

            # print(type(face_encoding), face_encoding);
            # print(type(self.known_faces[0]), self.known_faces[0]);

            tolerance = 0.60;
            # match = face_recognition.compare_faces(self.known_faces, face_encoding, tolerance=tolerance)
            match = face_recognition.face_distance(self.known_faces, face_encoding)

            # id = np.argmax(match);
            # name = self.known_names[id] if match[id] else None;

            id = np.argmin(match);
            personId, name = (self.known_names[id]) if (match[id] <= tolerance) else (None, None);

            people.append((id, personId, name));
            # print(people[-1]);

        for (top, right, bottom, left), name in zip(face_locations, people):
            name = name[2];

            color = (255, 0, 0) if name is None else (0, 255, 0);
            cv2.rectangle(img, (left, top), (right, bottom), color, 2);

            if(name is None): continue;

            cv2.rectangle(img, (left, bottom - 42), (right, bottom), color, cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(img, name, (left + 5, bottom - 8), font, 1.5, (0, 0, 0), 2);

        return img;

# video = FaceDetector(VideoReader(videoFile), facesDir);
# video.download(outputFile, 25);

# Main Classes for Local Setting

In [48]:
import numpy as np;
import cv2;
import termcolor;


class LocalVideoStream(VideoStreamBase):

    def displayVideo(self, displayName = 'Streaming'):
        while(True):
            img = self.getImage();
            if(img is None): break;

            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR);
            cv2.imshow(displayName, img);

            if(cv2.waitKey(1) & 0xFF == ord('q')): # press `q` to break
                break;


        print(termcolor.colored('video stream stopped voluntarily', 'green'));
        print(termcolor.colored('WARNING: object deleted', 'yellow'));
#         self.__del__();
        super(LocalVideoStream, self).__del__();


In [49]:
class LocalFaceDetector(LocalVideoStream, FaceDetector):
    def __init__(self, video: VideoStreamBase):
        super(LocalFaceDetector, self).__init__(video);


# Offilne Setting

In [50]:
import numpy as np
import cv2
import IPython
import PIL


class VideoReader(LocalVideoStream):

    def __init__(self, file):
        self.capture = cv2.VideoCapture(file)

    def __del__(self):
        self.capture.release()
        super(VideoReader, self).__del__();

    def __len__(self):
        return int(self.capture.get(cv2.CAP_PROP_FRAME_COUNT));

    def fps(self):
        return self.capture.get(cv2.CAP_PROP_FPS);

    def getImage(self):
        ret, frame = self.capture.read()
        if(ret == False): return None;

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        return frame;

# video = VideoReader(configs['video_file']);
# video.displayVideo();

In [ ]:
video = LocalFaceDetector(VideoReader(configs['video_file']));
video.download(configs['output_file']);

# Online Setting

In [ ]:
import numpy as np;
import requests;
import cv2;
from time import sleep;

class Camera(VideoStreamBase):

    def __init__(self, url):
        self.url = url;

    def getFrame(self):
        img_resp = requests.get(self.url);
        return img_resp;

    def getImage(self):
        img = self.getFrame();
        img = np.array(bytearray(img.content), dtype=np.uint8);

        img = cv2.imdecode(img, cv2.IMREAD_COLOR);
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        return img;


# camera = Camera(cameraUrl);
# camera.displayImage();

In [ ]:
# video = FaceDetector(Camera(cameraUrl), facesDir);
# video.download(outputFile, fps=10, limit=200, display=True);

# Laptop Webcam Setting

In [ ]:
import numpy as np;
import cv2;


class WebCam(LocalVideoStream):

    def __init__(self, deviceId: int = 0):
        self.capture = cv2.VideoCapture(deviceId);

    def Test(self):
        print('child');

#     def __del__(self):
#         print('releasing capture');
#         self.capture.release();
#         super(WebCam, self).__del__();

    def getImage(self):
        ret, img = self.capture.read();
        if(not ret): return None;

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img;

# video = WebCam();
# video.displayVideo();

In [ ]:
video = LocalFaceDetector(WebCam(), facesDir);
video.displayVideo();

# Local Setting

In [ ]:
class LocalCamera(LocalVideoStream, Camera):
    def __init__(self, url: str):
        super(LocalCamera, self).__init__(url);

# video = LocalCamera(cameraUrl);
# video.displayVideo();

In [ ]:
# video = LocalFaceDetector(LocalCamera(cameraUrl), facesDir);
# video.displayVideo();
